In [ ]:
!pwd

In [ ]:
from PIL import Image
import numpy as np


background = Image.open("../raw_data/facial_segmentation/image/0050_0001.jpg")

In [ ]:
foreground = Image.open("../raw_data/facial_segmentation/mask/0050_0001.png")

In [ ]:
R = np.array(background)[:,:,0]

In [ ]:
G = np.array(background)[:,:,1]

In [ ]:
B = np.array(background)[:,:,2]

In [ ]:
RGB = np.dstack(((R*np.array(foreground)),(G*np.array(foreground)) ,(B*np.array(foreground))))
RGB.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.array(background))

In [ ]:
set(np.array(foreground).flatten())

In [ ]:
everything_but_the_face = np.dstack((R*(np.array(foreground)==0),G*(np.array(foreground)==0) ,B*(np.array(foreground)==0)))
plt.imshow(everything_but_the_face )

In [ ]:
skin = np.dstack((R*(np.array(foreground)==1),G*(np.array(foreground)==1) ,B*(np.array(foreground)==1)))
plt.imshow(skin)

In [ ]:
right_brow = np.dstack((R*(np.array(foreground) ==2),G*(np.array(foreground) ==2) ,B*(np.array(foreground)==2)))

In [ ]:
left_brow = np.dstack((R*(np.array(foreground)==3),G*(np.array(foreground)==3) ,B*(np.array(foreground)==3)))

In [ ]:
brows = right_brow + left_brow
plt.imshow(brows)

In [ ]:
right_eye = np.dstack((R*(np.array(foreground)==4),G*(np.array(foreground)==4) ,B*(np.array(foreground)==4)))


In [ ]:
left_eye = np.dstack((R*(np.array(foreground)==5),G*(np.array(foreground)==5) ,B*(np.array(foreground)==5)))

In [ ]:
eyes = left_eye + right_eye
plt.imshow(eyes)

In [ ]:
lowerlip = np.dstack((R*(np.array(foreground)==7),G*(np.array(foreground)==7) ,B*(np.array(foreground)==7)))


In [ ]:
upperlip = np.dstack((R*(np.array(foreground)==9),G*(np.array(foreground)==9) ,B*(np.array(foreground)==9)))


In [ ]:
lips = upperlip + lowerlip
plt.imshow(lips)

In [ ]:
hair = np.dstack((R*(np.array(foreground)==10),G*(np.array(foreground)==10) ,B*(np.array(foreground)==10)))
plt.imshow(hair)

In [ ]:
plt.imshow(np.array(foreground)==10)

In [ ]:
type(hair)

In [ ]:
#face_features = [skin, hair, brows, lips ]


In [ ]:
## Flatten the segment to (num_pixels, 3) where 3 represents the RGB channels
#flat_segment = hair.reshape(-1,3)


In [ ]:
## Filter out zero pixels (background) by checking where any of the channels is non-zero
#non_zero_pixels = flat_segment[np.any(flat_segment != 0, axis=1)]

In [ ]:
## Calculate the mean for each channel
#mean_red = np.mean(non_zero_pixels[:, 0])
#mean_green = np.mean(non_zero_pixels[:, 1])
#mean_blue = np.mean(non_zero_pixels[:, 2])

In [ ]:
#mean_red

In [ ]:
def average_color(segment):
    """
    Calculate the average color of a segment ignoring zero values.

    Args:
    segment (numpy.ndarray): A 3D array (height, width, channels) representing the RGB values of a segment.

    Returns:
    tuple: A tuple representing the average (R, G, B) values of the segment.
    """
    # Flatten the segment to (num_pixels, 3) where 3 represents the RGB channels
    flat_segment = segment.reshape(-1, 3)

    # Filter out zero pixels (background) by checking where any of the channels is non-zero
    non_zero_pixels = flat_segment[np.any(flat_segment != 0, axis=1)]

    if non_zero_pixels.size == 0:
        # If no non-zero pixels are found, return a tuple with zeros (or any default value)
        return (0, 0, 0)

    # Calculate the mean for each channel
    mean_red = np.mean(non_zero_pixels[:, 0])
    mean_green = np.mean(non_zero_pixels[:, 1])
    mean_blue = np.mean(non_zero_pixels[:, 2])

    return (mean_red, mean_green, mean_blue)


In [ ]:
# Calculate the average color for each segment
average_skin_color = average_color(skin)
average_brows_color = average_color(brows)
average_eyes_color = average_color(eyes)
average_lips_color = average_color(lips)
average_hair_color = average_color(hair)

In [ ]:
average_skin_color

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def visualize_color(color, title):
    """
    Visualize a solid color image.

    Args:
    color (tuple): A tuple representing the RGB color (R, G, B).
    title (str): Title of the plot.
    """
    # Create a solid color image of size 100x100
    image = np.ones((100, 100, 3))
    image[:, :, 0] *= color[0] / 255.0
    image[:, :, 1] *= color[1] / 255.0
    image[:, :, 2] *= color[2] / 255.0

    # Plot the image
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()


In [ ]:
# Visualize colors
visualize_color(average_skin_color, 'Average Skin Color')
visualize_color(average_brows_color, 'Average Brows Color')
visualize_color(average_eyes_color, 'Average Eyes Color')
visualize_color(average_lips_color, 'Average Lips Color')
visualize_color(average_hair_color, 'Average Hair Color')

In [ ]:
def euclidean_distance(color1, color2):
    """
    Calculate the Euclidean distance between two RGB colors.

    Args:
    color1 (tuple): RGB values of the first color (R1, G1, B1).
    color2 (tuple): RGB values of the second color (R2, G2, B2).

    Returns:
    float: Euclidean distance between the two colors.
    """
    R1, G1, B1 = color1
    R2, G2, B2 = color2
    return np.sqrt((R1 - R2) ** 2 + (G1 - G2) ** 2 + (B1 - B2) ** 2)

def closest_color(target_color, color_palette):
    """
    Find the closest color from a palette to the target color using Euclidean distance.

    Args:
    target_color (tuple): RGB values of the target color.
    color_palette (list): List of RGB tuples representing the color palette.

    Returns:
    tuple: The closest RGB color from the palette.
    """
    closest = None
    min_distance = float('inf')
    for color in color_palette:
        distance = euclidean_distance(target_color, color)
        if distance < min_distance:
            min_distance = distance
            closest = color
    return closest

def analyze_palette(face_feature_color):
    """
    Analyzes and finds the closest color from predefined palettes to the given face feature color.

    Args:
    face_feature_color (tuple): RGB values of the face feature color.

    Returns:
    None: Prints the closest color from each palette and the closest overall palette.
    """
    # Define example color palettes (representing seasons)
    spring_palette = [
        (255, 223, 186),  # Light peach
        (255, 236, 179),  # Soft yellow
        (240, 191, 139),  # Coral
        (233, 141, 154),  # Light pink
    ]

    summer_palette = [
        (176, 224, 230),  # Light blue
        (250, 240, 230),  # Linen
        (224, 255, 255),  # Light cyan
        (255, 228, 225),  # Misty rose
    ]

    autumn_palette = [
        (139, 69, 19),    # Saddle brown
        (205, 133, 63),   # Peru
        (244, 164, 96),   # Sand brown
        (255, 165, 0),    # Orange
    ]

    winter_palette = [
        (0, 0, 139),      # Dark blue
        (255, 255, 255),  # White
        (128, 0, 0),      # Maroon
        (47, 79, 79),     # Dark slate gray
    ]

    # Find the closest color from each palette
    closest_spring_color = closest_color(face_feature_color, spring_palette)
    closest_summer_color = closest_color(face_feature_color, summer_palette)
    closest_autumn_color = closest_color(face_feature_color, autumn_palette)
    closest_winter_color = closest_color(face_feature_color, winter_palette)

    print("Closest Spring Color:", closest_spring_color)
    print("Closest Summer Color:", closest_summer_color)
    print("Closest Autumn Color:", closest_autumn_color)
    print("Closest Winter Color:", closest_winter_color)

    # Find the closest overall palette
    palette_distances = {
        "Spring": euclidean_distance(face_feature_color, closest_spring_color),
        "Summer": euclidean_distance(face_feature_color, closest_summer_color),
        "Autumn": euclidean_distance(face_feature_color, closest_autumn_color),
        "Winter": euclidean_distance(face_feature_color, closest_winter_color),
    }

    closest_palette = min(palette_distances, key=palette_distances.get)

    print("Closest Overall Palette:", closest_palette)

    # Visualize the closest palette
    if closest_palette == "Autumn":
        selected_palette = autumn_palette
    elif closest_palette == "Spring":
        selected_palette = spring_palette
    elif closest_palette == "Summer":
        selected_palette = summer_palette
    elif closest_palette == "Winter":
        selected_palette = winter_palette

    visualize_palette(selected_palette, f"Closest Palette: {closest_palette}")

def visualize_palette(palette, title="Color Wheel"):
    """
    Visualizes a given color palette as a color wheel.

    Args:
    palette (list): List of RGB tuples representing the color palette.
    title (str): Title for the plot.
    """
    num_colors = len(palette)
    angles = np.linspace(0, 2 * np.pi, num_colors, endpoint=False)
    fig, ax = plt.subplots(figsize=(5, 5), subplot_kw={'projection': 'polar'})

    for i, color in enumerate(palette):
        # Normalize color
        normalized_color = np.array(color) / 255.0
        # Plot each color in a segment of the wheel
        ax.bar(angles[i], 1, color=normalized_color, width=2*np.pi/num_colors)

    # Set title and remove axis labels
    ax.set_title(title, va='bottom')
    ax.set_yticklabels([])
    ax.set_xticklabels([])

    plt.show()


In [ ]:
#Skin color
analyze_palette(average_skin_color)

In [ ]:
analyze_palette(average_brows_color)

In [ ]:
analyze_palette(average_eyes_color)

In [ ]:
analyze_palette(average_lips_color)

In [ ]:
# KN clustering model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

def run_kmeans_clustering(colors, n_clusters):
    """
    Run K-Means clustering on a set of colors.

    Args:
    colors (np.ndarray): Array of RGB values (shape: num_colors x 3).
    n_clusters (int): Number of clusters to form.

    Returns:
    tuple: (kmeans model, cluster_centers, labels)
    """
    # Initialize K-Means with the desired number of clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)

    # Fit the model
    kmeans.fit(colors)

    # Get the cluster centers (average colors of each cluster)
    cluster_centers = kmeans.cluster_centers_

    # Get the labels for each color
    labels = kmeans.labels_

    return kmeans, cluster_centers, labels

# Example dataset: List of RGB colors (you can replace this with actual image data)
example_colors = np.array([
    [255, 0, 0],    # Red
    [0, 255, 0],    # Green
    [0, 0, 255],    # Blue
    [255, 255, 0],  # Yellow
    [255, 165, 0],  # Orange
    [128, 0, 128],  # Purple
    [0, 255, 255],  # Cyan
    [255, 255, 255], # White
    [255, 224, 189], # Skin tone
    [140, 70, 40],   # Brow color
    [30, 50, 100],   # Dark blue eye color
    [200, 80, 90],   # Light red lip color
    [50, 25, 10]     # Dark brown hair color
])

# Run K-Means clustering
n_clusters = 5  # Choose the number of clusters you want
kmeans, cluster_centers, labels = run_kmeans_clustering(example_colors, n_clusters)

# Display the results
print("Cluster Centers (Average Colors):", cluster_centers)

# Visualize the clusters
def visualize_clusters(cluster_centers):
    """
    Visualize the cluster centers as solid color images.

    Args:
    cluster_centers (np.ndarray): Array of RGB values for each cluster center.
    """
    n_centers = cluster_centers.shape[0]
    fig, ax = plt.subplots(1, n_centers, figsize=(15, 3))
    for i, color in enumerate(cluster_centers):
        ax[i].imshow(np.ones((100, 100, 3), dtype=np.uint8) * np.round(color).astype(np.uint8))
        ax[i].set_title(f"Cluster {i+1}")
        ax[i].axis('off')
    plt.show()

# Visualize the cluster centers
visualize_clusters(cluster_centers)